## Importing the Libraries

In [ ]:
import os
import glob
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#### Loading the raw loan acquisition txt file paths

In [22]:
loan_acquisition_file = glob.glob(os.path.join("Raw_data/Acquisition*.txt"))

In [23]:
#loading as a dataframe and the concatenating the files

df_from_each_file = (pd.read_csv(f,sep ="|", index_col=None, header=None) for f in loan_acquisition_file)
df = pd.concat(df_from_each_file, ignore_index=True)

In [24]:
#remaning the columns
df.rename(columns={
                    0: 'Loan_ID', 1: 'Channel', 2: 'Seller', 3: 'Interest_Rate', 4: 'UPB', 5: 'Loan_Term', 
                    6: 'Origination_Date', 7: 'First_Payment_Date', 8: 'LTV', 9: 'CLTV', 10: 'Num_Borrowers',
                    11: 'DTI', 12: 'Borrower_FICO', 13: 'First_Time_Buyer', 14: 'Loan_Purpose', 15: 'Dwelling_Type', 
                    16: 'Unit_Count', 17: 'Occupancy', 18: 'State', 19: 'Zip', 20: 'Insurance%', 21: 'Product',
                    22: 'Co_Borrower_FICO', 23: 'Mortgage_Insurance_Type',24: 'Relocation_Indicator'}, 
                    inplace=True)

In [25]:
#saving the file locally
df.to_csv('Acquisition_Q3_merged.csv',index=False)

#### Loading the raw loan performance txt file paths


In [26]:

loan_performance_file = glob.glob(os.path.join( "Raw_data/Performance_*.txt"))

In [27]:
# preprocessing function which take the latest loan_id row

def processing(df):

    df['MONTHLY  REPORTING  PERIOD'] = df['MONTHLY  REPORTING  PERIOD'].apply(str).str[6:].apply(int)*100+df['MONTHLY  REPORTING  PERIOD'].apply(str).str[:2].apply(int)

    idx = df.groupby(['LOAN IDENTIFIER'])['MONTHLY  REPORTING  PERIOD'].transform(max) == df['MONTHLY  REPORTING  PERIOD']

    df = df[idx].copy()
    
    return df


In [ ]:
#loading as a dataframe, preprocess it an then finally save it as a csv files.

for file in loan_performance_file:
    cols = ['LOAN IDENTIFIER', 'MONTHLY  REPORTING  PERIOD', 'SERVICER  NAME', 'currIntRate', 'CURRENT  ACTUAL  UPB',
            'LOAN AGE', 'REMAINING  MONTHS  TO  LEGAL MATURITY', 'ADJUSTED  MONTHS  TO  MATURITY', 'MATURITY  DATE', 
            'METROPOLITAN  STATISTICAL  AREA  (MSA)', 'CURRENT  LOAN  DELINQUENCY  STATUS', 'MODIFICATION  FLAG',
            'ZERO BALANCE CODE', 'ZERO  BALANCE  EFFECTIVE  DATE', 'LAST PAID INSTALLMENT  DATE', 'FORECLOSURE  DATE',
            'DISPOSITION  DATE', 'FORECLOSURE  COSTS', 'PROPERTY  PRESERVATION  AND REPAIR  COSTS', 'ASSET  RECOVERY  COSTS',
            'MISCELLANEOUS  HOLDING EXPENSES  AND  CREDITS' , 'ASSOCIATED  TAXES  FOR HOLDING PROPERTY', 'NET  SALE PROCEEDS',
            'CREDIT ENHANCEMENT   PROCEEDS', 'REPURCHASE  MAKE  WHOLE  PROCEEDS', 'OTHER  FORECLOSURE  PROCEEDS', 
            'NON  INTEREST  BEARING  UPB', 'PRINCIPAL FORGIVENESS   AMOUNT', 'REPURCHASE  MAKE  WHOLE  PROCEEDS  FLAG',
            'FORECLOSURE  PRINCIPAL WRITE-OFF  AMOUNT', 'SERVICING  ACTIVITY  INDICATOR']
    
    df = pd.read_csv(file, sep='|', index_col=None, header=None, names=cols)  
    df = processing(df)
    file_name = file.split('.')[0] + '_processed.csv'
    df.to_csv(file_name,index=False)    

#### Loading the preprocessed raw loan performance csv file paths


In [29]:
loan_performance_file = glob.glob(os.path.join( "raw_data/Performance_*.csv"))

In [30]:
#loading as a dataframe and the concatenating the files

df_from_each_file = (pd.read_csv(f) for f in loan_performance_file)
perf_df   = pd.concat(df_from_each_file, ignore_index=True)
perf_df.to_csv('Performace_Q3_merged.csv',index=False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DtypeWarning: Columns (14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  


## Merging the loan performace and acquisition file

In [75]:
perf_merged, acq_merged = pd.read_csv('Performace_Q3_merged.csv'), pd.read_csv('Acquisition_Q3_merged.csv')

In [76]:

loan_data = acq_merged.merge(perf_merged , left_on='Loan_ID', right_on='LOAN IDENTIFIER', how='inner')
loan_data.drop('LOAN IDENTIFIER',inplace=True,axis=1)
loan_data.to_csv('load_data_Q3.csv',index=False)

In [153]:
loan_data = pd.read_csv('load_data_Q3.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [157]:
#mapping the zero code balance into three categories under current_status column

loan_data['ZERO BALANCE CODE'].fillna(0,inplace=True)

zero_bal_cd_map = {0:'Current',1:'Prepaid',2:'Third Party Sale',3:'Short Sale',
                   6:'Repurchase',9:'REO',15:'Note Sale',16:'RPL Loan Sale'}

loan_data['ZERO BALANCE CODE'] = loan_data['ZERO BALANCE CODE'].map(zero_bal_cd_map).apply(str)

zero_bal_cd_map = {'Current':'Current','Prepaid':'Prepaid','Third Party Sale':'Underperforming','Short Sale':'Underperforming',
                   'Repurchase':'Underperforming','REO':'Underperforming','Note Sale':'Underperforming','RPL Loan Sale':'Underperforming'}

loan_data['Current_Status'] = loan_data['ZERO BALANCE CODE'].map(zero_bal_cd_map).apply(str)

In [158]:
#filling up the null values

loan_data['Mortgage_Insurance_Type'].fillna(0,inplace=True)
loan_data['Insurance%'].fillna(0,inplace=True)
loan_data['Num_Borrowers'].fillna(1,inplace=True)
loan_data['CLTV'].fillna(loan_data['LTV'],inplace=True)
#loan_data.drop('Co_Borrower_FICO',axis=1,inplace=True)

In [159]:
#Binning the fico score and loan term

FICO_bins = [0,620,660,700,740,780,850]
FICO_labels = ['0-620', '620-660','660-700','700-740','740-780','780+']
loan_data['FICO_bins'] = pd.cut(loan_data['Borrower_FICO'],bins=FICO_bins,labels=FICO_labels)

Term_bins =[0,180,360]
Term_labels =['<=15 Years','<= 30 Years']
loan_data['Term_bins'] = pd.cut(loan_data['Loan_Term'],bins=Term_bins,labels=Term_labels)

#loan_data.drop(['Borrower_FICO','Loan_Term'],inplace=True,axis=1)

In [160]:
#filling the loan delinquency and forclosure date

loan_data['FORECLOSURE  DATE'].fillna(-1,inplace=True)
loan_data['CURRENT  LOAN  DELINQUENCY  STATUS'] = loan_data['CURRENT  LOAN  DELINQUENCY  STATUS'].replace(np.NaN,0)
loan_data['CURRENT  LOAN  DELINQUENCY  STATUS'] = loan_data['CURRENT  LOAN  DELINQUENCY  STATUS'].replace('X',-1)
loan_data['CURRENT  LOAN  DELINQUENCY  STATUS'] = loan_data['CURRENT  LOAN  DELINQUENCY  STATUS'].astype('int64')

In [161]:
def code_delinq(value):
    
    if (int(value)<30):
        value = 0
    
    if (int(value) >= 30 and int(value) <= 89):
        value = 1
        
    if (int(value) > 89):
        value = 2
        
    return value

loan_data['CurrentLoanDeliquencyStatus_map'] = loan_data['CURRENT  LOAN  DELINQUENCY  STATUS'].apply(code_delinq)


In [162]:
#mapping the CurrentLoanDeliquencyStatus into three categories under CurrentLoanDeliquencyStatus_map column
CurrentLoanDeliquencyStatus_map = {0:'low',1:'Moderate',2:'High'}

loan_data['CurrentLoanDeliquencyStatus_map'] = loan_data['CurrentLoanDeliquencyStatus_map'].map(CurrentLoanDeliquencyStatus_map).apply(str)

In [163]:
loan_data['CurrentLoanDeliquencyStatus_map'].value_counts()

low         2081316
Moderate        188
High             62
Name: CurrentLoanDeliquencyStatus_map, dtype: int64

In [164]:
loan_data['Current_Status'].value_counts()

Prepaid            1096625
Current             949530
Underperforming      35411
Name: Current_Status, dtype: int64

In [165]:
loan_data['FORECLOSURE  DATE'].value_counts()

-1            2053662
09/01/2010        526
07/01/2010        504
06/01/2010        482
08/01/2010        480
               ...   
01/01/2009          8
03/01/2005          7
01/01/2005          2
02/01/2005          2
12/01/2004          1
Name: FORECLOSURE  DATE, Length: 176, dtype: int64

In [166]:
zero_bal_cd_map = {0:'Current',1:'Prepaid',2:'Third Party Sale',3:'Short Sale',
                   6:'Repurchase',9:'REO',15:'Note Sale',16:'RPL Loan Sale'}

zero_bal_cd_map_ = {'Current':'Current','Prepaid':'Prepaid','Third Party Sale':'Underperforming','Short Sale':'Underperforming',
                   'Repurchase':'Underperforming','REO':'Underperforming','Note Sale':'Underperforming','RPL Loan Sale':'Underperforming'}

In [167]:
def target(currentstatus,Delinquent_Ind,Foreclosure_Ind):
    if(Foreclosure_Ind!=-1):
        val='High-Risk'
    elif (currentstatus=='Current' and Delinquent_Ind=='High') or (currentstatus =='Underperforming'):
        val='High-Risk'
    elif (currentstatus=='Current' and Delinquent_Ind=='Moderate') or (currentstatus=='Prepaid'):
        val='Moderate-Risk'
    else:
        val='No-Risk'
        
    return val

In [168]:
loan_data['target'] = loan_data.apply(lambda row: target(row['Current_Status'], row['CURRENT  LOAN  DELINQUENCY  STATUS'],
                                               row['FORECLOSURE  DATE']), axis=1)


In [169]:
loan_data['target'].value_counts()

Moderate-Risk    1096625
No-Risk           949530
High-Risk          35411
Name: target, dtype: int64

In [170]:
loan_data['target'].value_counts()

Moderate-Risk    1096625
No-Risk           949530
High-Risk          35411
Name: target, dtype: int64

In [171]:
loan_data['Origin_Month'], loan_data['Origin_Year'] = loan_data['First_Payment_Date'].str.split('/', 1).str

loan_data = loan_data[loan_data['Origin_Year'].isin(['2004','2008','2012','2016'])]

In [172]:
loan_data.to_csv('processed_loan_origin.csv',index=False)

In [174]:
loan_data = pd.read_csv('processed_loan_origin.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [230]:
summary = {'Loan_ID':'count',
     'UPB':"mean",
     'Borrower_FICO': 'mean',
     'Co_Borrower_FICO': 'mean',
     'LTV': 'mean',
     'CLTV': 'mean',
     'DTI': 'mean',
     'Interest_Rate': 'mean'}

expenses = {'FORECLOSURE  COSTS':'mean',
'PROPERTY  PRESERVATION  AND REPAIR  COSTS':'mean',
'ASSET  RECOVERY  COSTS':'mean',
'MISCELLANEOUS  HOLDING EXPENSES  AND  CREDITS':'mean',
'ASSOCIATED  TAXES  FOR HOLDING PROPERTY':'mean'}

proceeds = {'NET  SALE PROCEEDS':'mean',
'CREDIT ENHANCEMENT   PROCEEDS':'mean',
'REPURCHASE  MAKE  WHOLE  PROCEEDS':'mean',
'OTHER  FORECLOSURE  PROCEEDS':'mean',
'OTHER  FORECLOSURE  PROCEEDS':'mean'}

In [231]:
acq_summary = loan_data.groupby(['Origin_Year']).agg(summary).round(2)
acq_summary.index.name = ['Year_Q3']
acq_summary.columns = ['Loan Count', 'Average Unpaid Balance', 'Borrower Credit Score', 'Co-Borrower Credit Score', 'LTV Ratio', 'CLTV Ratio', 'DTI', 'Interest Rate']


In [232]:
exp_summaray = loan_data.groupby(['Origin_Year']).agg(expenses).round(2)
exp_summaray.index.name = ['Year_Q3']
exp_summaray.columns = ['Avg. Foreclosure  Cost', 'Avg. Property Preservation', 'Avg. Asset Recovery ', 'Avg. Misc. Holding Expenses/Credits', 'Avg. Associated Taxes']


In [233]:
proc_summary = loan_data.groupby(['Origin_Year']).agg(proceeds).round(2)
proc_summary.index.name = ['Year_Q3']
proc_summary.columns = ['Avg. Net Sales Proceeds', 'Avg. Credit Enhancement', 'Avg . Repurchase/Make Whole ', 'Avg. Other Proceeds']


In [279]:
status_summary = loan_data.groupby(['Origin_Year','Current_Status']).agg({'Loan_ID':'count'})

status_summary = status_summary.reset_index()
status_summary.columns = ['[Year_Q3]','Loan Status','Loan Count']
status_summary = status_summary.groupby(['[Year_Q3]','Loan Status']).sum()

In [280]:
modifi_summary = loan_data.groupby(['Origin_Year','MODIFICATION  FLAG']).agg({'Loan_ID':'count'})

modifi_summary = modifi_summary.reset_index()

modifi_summary.columns = ['[Year_Q3]','Modification Flag','Flag Count']
modifi_summary = modifi_summary.groupby(['[Year_Q3]','Modification Flag']).sum()


In [281]:
acq_summary

,Loan Count,Average Unpaid Balance,Borrower Credit Score,Co-Borrower Credit Score,LTV Ratio,CLTV Ratio,DTI,Interest Rate
[Year_Q3],,,,,,,,
2004,384600,153335.04,719.75,727.34,70.01,71.37,36.03,6.02
2008,352362,208150.30,741.05,750.33,72.44,73.44,38.10,6.30
2012,715201,229735.46,770.49,775.58,69.07,70.05,30.70,3.65
2016,623054,236064.10,757.61,763.96,72.98,73.50,32.79,3.66


In [282]:
exp_summaray

,Avg. Foreclosure Cost,Avg. Property Preservation,Avg. Asset Recovery,Avg. Misc. Holding Expenses/Credits,Avg. Associated Taxes
[Year_Q3],,,,,
2004,5662.29,5284.41,1525.38,1790.74,5431.05
2008,5696.84,6127.33,1708.71,2119.50,6407.52
2012,6171.62,8061.31,2209.89,1455.22,5263.31
2016,5376.64,6549.92,2134.00,1401.99,2840.58


In [283]:
proc_summary

,Avg. Net Sales Proceeds,Avg. Credit Enhancement,Avg . Repurchase/Make Whole,Avg. Other Proceeds
[Year_Q3],,,,
2004,92193.51,34970.34,68484.45,5287.44
2008,124223.85,49609.77,131980.17,4851.51
2012,129173.91,44422.67,153919.16,6517.94
2016,151865.99,31239.89,12477.67,4569.47


In [284]:
status_summary

Loan Count
[Year_Q3] Loan Status                
2004      Current               25349
          Prepaid              345991
          Underperforming       13260
2008      Current               25552
          Prepaid              306336
          Underperforming       20474
2012      Current              383469
          Prepaid              330717
          Underperforming        1015
2016      Current              514376
          Prepaid              108127
          Underperforming         551

In [285]:
modifi_summary

Flag Count
[Year_Q3] Modification Flag            
2004      N                      375067
          Y                        9533
2008      N                      336072
          Y                       16290
2012      N                      713144
          Y                        2057
2016      N                      620674
          Y                        2380